In [5]:
import pandas as pd
import numpy as np

Считываем данные: я считываю 1 процент от всех рандомно ради ускорения подсчётов. И сразу дропаем ненужные по условию колонки.

In [6]:
data = pd.read_csv('../data/data.csv').sample(frac=0.01, random_state=1337)
data.drop(['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'], axis=1, inplace=True)
data.head()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks
741246,2021-09-29 18:06:03.000000,13,358,0,2,5,1,0
11133560,2021-09-26 02:04:47.000000,0,3,0,2,1,1,0
5348767,2021-09-28 00:26:59.000000,15,16,16,0,6,1,0
4078254,2021-09-26 17:07:20.000000,17,59,0,2,7,1,0
8784861,2021-09-29 15:47:26.000000,17,22,0,2,7,1,0


Проверяем есть в данных нулы или нет, есть ли данные, которые не были показаны и смотрим достаточно ли хорошо мы рандомно взяли данные по дням: для этого приводим дату к `datetime`.

In [7]:
def analysis(data: pd.DataFrame):
    print(f"Any nulls in data={data.isnull().values.any()}")
    print(f"Is there not impressed ad={(data['impressions'] == 0).any()}")
    print("Number of rows in different days")
    data['date_time'] = pd.to_datetime(data['date_time'])
    print(data.groupby([data['date_time'].dt.date]).count())

analysis(data)

Any nulls in data=False
Is there not impressed ad=False
Number of rows in different days
            date_time  zone_id  banner_id  campaign_clicks  os_id  country_id  \
date_time                                                                       
2021-09-26      30924    30924      30924            30924  30924       30924   
2021-09-27      23708    23708      23708            23708  23708       23708   
2021-09-28      22882    22882      22882            22882  22882       22882   
2021-09-29      24498    24498      24498            24498  24498       24498   
2021-09-30      18502    18502      18502            18502  18502       18502   
2021-10-01      16371    16371      16371            16371  16371       16371   
2021-10-02      21330    21330      21330            21330  21330       21330   

            impressions  clicks  
date_time                        
2021-09-26        30924   30924  
2021-09-27        23708   23708  
2021-09-28        22882   22882  
2021-09-29 

По результатам анализа удаляем impressions. Также я удалил campaign_clicks, потому что по-моему это ни о чём не говорит, так как пользователь мог и не заметить то, что мы ему что-то показываем, а также для удобства, так как это единственная некатегориальная фича и мы в итоге побили бейзлайн и без неё.

In [8]:
from sklearn.preprocessing import OneHotEncoder

def feature_engineering(data: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):
    data.drop(['campaign_clicks', 'impressions'], axis=1, inplace=True)
    train = data[data['date_time'] < '2021-10-02']
    train_clicks = train['clicks']
    test = data[data['date_time'] >= '2021-10-02']
    test_clicks = test['clicks']
    train = train.drop(['date_time', 'clicks'], axis=1)
    test = test.drop(['date_time', 'clicks'], axis=1)
    ohe = OneHotEncoder(handle_unknown='ignore')
    train = ohe.fit_transform(train)
    test = ohe.transform(test)
    return train, train_clicks, test, test_clicks

train_x, train_y, test_x, test_y = feature_engineering(data)

Проверяем, что все данные имеют ожидаемый размер.

In [9]:
print(data.head())
print(data.shape)
print(train_y.shape)
print(train_x.shape)

                   date_time  zone_id  banner_id  os_id  country_id  clicks
741246   2021-09-29 18:06:03       13        358      2           5       0
11133560 2021-09-26 02:04:47        0          3      2           1       0
5348767  2021-09-28 00:26:59       15         16      0           6       0
4078254  2021-09-26 17:07:20       17         59      2           7       0
8784861  2021-09-29 15:47:26       17         22      2           7       0
(158215, 6)
(136885,)
(136885, 2184)


Используем LogisticRegression, который упоминался в лекции, вместе с liblinear - не SGD-like оптимизатором.

In [10]:
from sklearn.linear_model import LogisticRegression

def create_model(penalty, C):
    return LogisticRegression(C = C, penalty=penalty, solver='liblinear')

Были эксперименты с l1, но без каких-либо приростов, поэтому в итоговом прогоне решил его не оставлять. Как видно, C=1 показал лучшие метрики

In [11]:
from sklearn.model_selection import cross_validate

def cv(x, y):
    Cs = [0.001, 0.01, 0.1, 1, 10, 100]
    for penalty in ['l2']:
        for C in Cs:
            model = create_model(penalty, C)
            scores = cross_validate(model, x, y, scoring=['neg_log_loss', 'roc_auc'], cv=5)
            print(f"C={C}, neg_los_loss={np.mean(scores['test_neg_log_loss'])}, roc_auc={np.mean(scores['test_roc_auc'])}, penalty={penalty}")

cv(train_x, train_y)

C=0.001, neg_los_loss=-0.11995052836774181, roc_auc=0.6227093187693808, penalty=l2
C=0.01, neg_los_loss=-0.10993894859474289, roc_auc=0.7025020493358, penalty=l2
C=0.1, neg_los_loss=-0.10641176293498181, roc_auc=0.7363618610540742, penalty=l2
C=1, neg_los_loss=-0.10591685029366242, roc_auc=0.7390292097447666, penalty=l2
C=10, neg_los_loss=-0.10885617620559566, roc_auc=0.7227832760412876, penalty=l2
C=100, neg_los_loss=-0.11366192945983244, roc_auc=0.710983583716881, penalty=l2


Сравниваем нашу лучшую модель с бейзлайном, чтобы удостовериться, что мы лучше.

In [12]:
from sklearn.metrics import roc_auc_score, log_loss

model = create_model(penalty='l2', C=1).fit(train_x, train_y)
y_pred_proba = model.predict_proba(test_x)[:, 1]
roc_auc_metric = roc_auc_score(test_y, y_pred_proba)
log_loss_metric = log_loss(test_y, y_pred_proba)
print(f"roc_auc={roc_auc_metric}, log_loss={log_loss_metric}")

y_pred_base = np.full(y_pred_proba.shape, np.mean(train_y))
roc_auc_metric_base = roc_auc_score(test_y, y_pred_base)
log_loss_metric_base = log_loss(test_y, y_pred_base)
print(f"roc_auc_base: {roc_auc_metric_base}, log_loss base: {log_loss_metric_base}")

roc_auc=0.7523089444288413, log_loss=0.14680601301522303
roc_auc_base: 0.5, log_loss base: 0.16643236596578298
